### Amazon Bedrock Prompt Flows
---

Amazon Bedrock Prompt flows offers the ability for you to use supported foundation models (FMs) to build workflows by linking prompts, foundational models, and other AWS services to create end-to-end solutions.

With prompt flows, you can quickly build complex generative AI workflows using a visual builder, easily integrate with Amazon Bedrock offerings such as FMs, knowledge bases, and other AWS services such as AWS Lambda by transferring data between them, and deploying immutable workflows to move from testing to production in few clicks.

#### Create the prompt flow:
1. Specify a prompt flow name, description, and appropriate IAM permissions.

2. Design your prompt flow by deciding the nodes you want to use.

3. Create or define all the resources you require for each node. For example, if you are planning to use an AWS Lambda function, define the functions you need for the node to complete its task.

4. Add nodes to your prompt flow, configure them, and create connections between the nodes by linking the output of a node to the input of another node in the prompt flow.

#### Test the prompt flow:
1. Prepare the prompt flow, so that the latest changes apply to the working draft of the prompt flow, a version of the prompt flow that you can use to iteratively test and update your prompt flow

2. Test the prompt flow by invoking it with sample inputs to see the outputs it yields.

3. When you're satisfied with a prompt flow's configuration, you can create a snapshot of it by publishing a version. The version preserves prompt flow definition as it exists at the time of the creation. Versions are immutable because they act as a snapshot of the prompt flow at the time it was created.

#### Deploy the prompt flow
1. Create an alias that points to the version of your prompt flow that you want to use in your application.

2. Set up your application to make InvokeFlow requests to the alias. If you need to revert to an older version or upgrade to a newer one, you can change the routing configuration of the alias.

In [ ]:
!pip install -Uq boto3 botocore

In [ ]:
import json
import boto3
from datetime import datetime

BEDROCK_HAIKU_MODELID: str = "anthropic.claude-3-haiku-20240307-v1:0"
# The prompt management and flows features are part of the bedrock agent SDK
bedrock_agent = boto3.client(service_name="bedrock-agent", region_name="us-east-1")

_Amazon Bedrock Prompt Management_ streamlines the creation, evaluation, deployment, and sharing of prompts in the Amazon Bedrock console and via APIs in the SDK. This feature helps developers and business users obtain the best responses from foundation models for their specific use cases.

In [ ]:
prompt_template: str = """You're a customer service agent for the e-commerce company Octank. 
                    Answer the following user query in a friendly and direct way: {{input}}"""

In [ ]:
response = bedrock_agent.create_prompt(
    name = f"MyTestPrompt-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    description = "This is my test prompt for the customer service use case",
    variants = [
        {
            "inferenceConfiguration": {
                "text": {
                    "maxTokens": 3000,
                    "temperature": 0,
                    "topP": 0.999,
                }
            },
            "modelId": BEDROCK_HAIKU_MODELID,
            "name": "variant-001",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {
                            "name": "input"
                        }

                    ],
                    "text": prompt_template
                }
            },
            "templateType": "TEXT"
        }
    ],
    defaultVariant = "variant-001"
)
print(json.dumps(response, indent=2, default=str))
promptId = response["id"]
promptArn = response["arn"]
promptName = response["name"]
print(f"Prompt ID: {promptId}\nPrompt ARN: {promptArn}\nPrompt Name: {promptName}")

In [ ]:
# now that we have a draft prompt, we can create versions from it
response = bedrock_agent.create_prompt_version(
    promptIdentifier = promptId
)
print(json.dumps(response, indent=2, default=str))

In [ ]:
# list of prompts in our Prompt Library or catalog
response = bedrock_agent.list_prompts(
    maxResults = 10
)
print(json.dumps(response["promptSummaries"], indent=2, default=str))

In [ ]:
# read details about a prompt
response = bedrock_agent.get_prompt(
    promptIdentifier = promptId,
    promptVersion = "1"
)
print(json.dumps(response, indent=2, default=str))

### Prompt Flows
Now that we've learned how to create and manage prompts, we can continue exploring how to build generative AI applications logic by creating workflows. For this, we'll rely on Prompt Flows for Amazon Bedrock.

### Create and Manage Flows
Let's create a simple flow that will load a prompt from our catalog. Note you can also create more complex flows involving chaining of steps, and conditions for dynamically routing, but let's keep it simple for now.

_Pre-requisite: For using Flows you need to make sure you have the proper AWS IAM permissions in place. You can check details in the How Prompt Flows for Amazon Bedrock works documentation._

In [ ]:
flow_role: str = "arn:aws:iam::015469603702:role/prompt_flow_role"

In [ ]:
response = bedrock_agent.create_flow(
    name = f"MyTestFlow-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    description = "This is my test flow for the customer service use case",
    executionRoleArn = flow_role,
    definition = {
      "nodes": [
          {
              "name": "StartNode",
              "type": "Input",
              "configuration": {
                  "input": {}
              },
              "outputs": [
                  {
                      "name": "document",
                      "type": "String"
                  }
              ],
          },
          {
            "name": "Prompt_1",
            "type": "Prompt",
            "configuration": {
              "prompt": {
                "sourceConfiguration": {
                  "resource": {
                      "promptArn": promptArn
                  }
                }
              }
            },
            "inputs": [
              {
                "expression": "$.data",
                "name": "input",
                "type": "String"
              }
            ],
            "outputs": [
              {
                "name": "modelCompletion",
                "type": "String"
              }
            ],
          },
          {
            "name": "EndNode",
            "type": "Output",
            "configuration": {
                "output": {}
            },
            "inputs": [
              {
                "expression": "$.data",
                "name": "document",
                "type": "String"
              }
            ],
          }
      ],
      "connections": [
          {
              "name": "Connection_1",
              "source": "StartNode",
              "target": "Prompt_1",
              "type": "Data",
              "configuration":{
                  "data": {
                      "sourceOutput": "document",
                      "targetInput": "input"
                  }
              }
          },
          {
              "name": "Connection_2",
              "source": "Prompt_1",
              "target": "EndNode",
              "type": "Data",
              "configuration": {
                  "data": {
                      "sourceOutput": "modelCompletion",
                      "targetInput": "document"
                  }
              }
          }
      ],
    }
)
print(json.dumps(response, indent=2, default=str))
flowId = response["id"]
flowArn = response["arn"]
flowName = response["name"]
print(f"Flow ID: {flowId}\nFlow ARN: {flowArn}\nFlow Name: {flowName}")

In [ ]:
# Now that we have our first flow, we can prepare it. This basically builds and validates our flow.
response = bedrock_agent.prepare_flow(
    flowIdentifier = flowId
)
print(json.dumps(response, indent=2, default=str))

In [ ]:
response = bedrock_agent.get_flow(
    flowIdentifier = flowId
)
print(json.dumps(response, indent=2, default=str))

In [ ]:
response = bedrock_agent.create_flow_version(
    flowIdentifier = flowId
)
print(json.dumps(response, indent=2, default=str))

In [ ]:
response = bedrock_agent.create_flow_alias(
    flowIdentifier = flowId,
    name = flowName,
    description = "Alias for my test flow in the customer service use case",
    routingConfiguration = [
        {
            "flowVersion": "1"
        }
    ]
)
print(json.dumps(response, indent=2, default=str))
flowAliasId = response['id']

In [ ]:
response = bedrock_agent.update_flow_alias(
    flowIdentifier = flowId,
    aliasIdentifier = flowAliasId,
    name = flowName,
    routingConfiguration = [
        {
            "flowVersion": "1"
        }
    ]
)
flowAliasId = response["id"]
print(json.dumps(response, indent=2, default=str))

In [ ]:
bedrock_agent_runtime = boto3.client(service_name = 'bedrock-agent-runtime', region_name = 'us-east-1')
# invoke a flow
response = bedrock_agent_runtime.invoke_flow(
    flowIdentifier = flowId,
    flowAliasIdentifier = flowAliasId,
    inputs = [
        { 
            "content": { 
                "document": "Hi, I need help with my order!"
            },
            "nodeName": "StartNode",
            "nodeOutputName": "document"
        }
    ]
)
event_stream = response["responseStream"]
for event in event_stream:
    print(json.dumps(event, indent=2, ensure_ascii=False))